In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install git+https://github.com/openai/whisper.git
! pip install git+https://github.com/pytube/pytube.git
! pip install pandas
!pip install googletrans==4.0.0-rc1

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-dzcz8x0u
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-dzcz8x0u
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802824 sha256=ae00f6ad397950fe7368c8b76bf13c9adc6f848da9cbeba54172f34cebf44dfa
  Stored in directory: /tmp/pip-ephem-wheel-cache-hb1kt8xt/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
  Cloning https://github.com/pytube/pytube.git to /tmp/pip-req-build-igoz9i5s


In [3]:
import os
import whisper
from googletrans import Translator
import csv
import datetime

# Initialize Whisper and Translator models
model = whisper.load_model("large")
translator = Translator()

# Directory path where audio files are stored
audio_dir = "/content/drive/MyDrive/genai data/"

# Function to process audio files, transcribe, and translate if necessary
def process_audio_files(audio_dir):
    transcription_data = []

    # Iterate over audio files in the directory
    for file_name in os.listdir(audio_dir):
        if file_name.endswith(".mp3") or file_name.endswith(".mp4"):
            file_path = os.path.join(audio_dir, file_name)

            # Transcribe the audio
            transcription = model.transcribe(file_path)

            if (transcription["text"]) != "":

              # Check if the transcription is in Urdu and translate it
              if detect_language(transcription["text"]) == "ur":
                  translated_text = translator.translate(transcription["text"], src='ur', dest='en').text

              elif detect_language(transcription["text"]) == "hi":
                  translated_text = translator.translate(transcription["text"], src='hi', dest='en').text

              else:
                  translated_text = transcription["text"]

              print(translated_text)

              # Chunk transcription and collect metadata
              chunks = chunk_transcription(translated_text)
              metadata = index_metadata(file_name, transcription, chunks)

              transcription_data.append({
                  "audio_file": file_name,
                  "transcription": transcription["text"],
                  "translated_text": translated_text,
                  "chunks": chunks,
                  "metadata": metadata
              })

    return transcription_data

# Function to detect language (mocked for simplicity)
def detect_language(text):
  try:
    # Detect the language of the text
    detection = translator.detect(text)

    # Check if the detected language is Urdu (code: 'ur')
    if detection is not None & detection.lang == 'ur':
        return "ur"
    elif detection is not None & detection.lang == 'hi':
        return "hi"
    else:
        return "en"
  except:
    return ""

# Function to chunk transcription into manageable parts
def chunk_transcription(text, chunk_size=100):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Function to index transcription with metadata (timestamps and file link)
def index_metadata(file_name, transcription, chunks):
    indexed_data = []

    for i, chunk in enumerate(chunks):
        start_time = i * (len(transcription["segments"]) // len(chunks))
        end_time = (i + 1) * (len(transcription["segments"]) // len(chunks))

        indexed_data.append({
            "chunk": chunk,
            "audio_file": file_name,
            "start_time": start_time,
            "end_time": end_time
        })

    return indexed_data

# Function to save the transcription data to CSV
def save_to_csv(transcription_data, csv_file_name="transcriptions.csv"):
    # Define the CSV file headers
    headers = ["audio_file", "original_transcription", "translated_transcription", "chunk", "start_time", "end_time"]

    # Open CSV file and write data
    with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()

        # Write each chunk of data to CSV
        for item in transcription_data:
            for i, chunk in enumerate(item["chunks"]):
                # Write each chunk along with its metadata to CSV
                writer.writerow({
                    "audio_file": item["audio_file"],
                    "original_transcription": item["transcription"],
                    "translated_transcription": item["translated_text"],
                    "chunk": chunk,
                    "start_time": item["metadata"][i]["start_time"],
                    "end_time": item["metadata"][i]["end_time"]
                })

# Example usage
data = process_audio_files(audio_dir)
print (data)

csv_filename = audio_dir + "transcriptions" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")+".csv"

# Save the data to a CSV file
save_to_csv(data, csv_file_name= csv_filename)

100%|█████████████████████████████████████| 2.88G/2.88G [00:34<00:00, 88.4MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

 स्मार्ट एलेडी टीवी How to select input sources by remote control Remote control में मौझूद source बटन को क्लिक करें Source menu screen पर appear हो जाएगा मौझूद options के features को जानें Home इस option के जरिये आप YouTube, Netflix, Prime Video तमाम Android apps और Media Player for USB access को use कर सकते हैं ATV इस feature के जरिये आप लगाएगाई cable connection के थूँ analog channels देख सकते हैं DVB-T, T2 जिसमें terrestrial channels को देख सकते हैं ये feature पाकिस्तान में मौझूद नहीं है DVBC के जरिये आप लगाएगाई cable connection के थूँ digital channels देख सकते हैं smart LED TV में audio और video दोनों को connect कर सकते हैं इसमें मौझूद 3 HDMI connection options भी हैं जिसके जरिये आप अपने computer, laptop, game, DVD player जैसे gadget से भी अपने smart LED TV को connect कर सकते हैं अपनी जरूरत के मताबे कोई भी option select करें और OK का button दबा दें अपनी जरूरत के मताबे को connect कर दें


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 स्मार्ट एलेडी टीवी How to use smart LED TV menu function रिमोर्ट कंट्रोल में मौझूद menu के बटन को प्रेस करें पिच्चर या साउंड सेटिंग के ओप्शन को एजजस करने के लिए more के ओप्शन को स्लेक्ट करें रिमोर्ट कंट्रोल पर दिये गए up and down एरोस के जरिए पिच्चर सेटिंग को स्लेक्ट करें और OK का बटन दबा करें पिच्चर सेटिंग में मौझूद brightness, contrast, saturation, sharpness, gamma को अपनी जरूरत के मताबग एजजस करें साउंड ओप्शन को स्लेक्ट करें और साउं Protect को अपनी जरूरत के मताबग एजजस करें चैनल को ट्यून करने के लिए टीवी को ऑप्शन सलेक्ट करें सेटअप ऑप्शन के जरिए अपनी जरूरत के मताबग सेटिंग को एजजस करें सेटिंग को ऑप्शन को सलेक्ट करके LED TV सेटिंग को एजजस करें


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

 समार्ट एलेडी टीवी सबसे पहले रिमोर्ट कंट्रोल पर दिये गए होम बटन पर क्लिक करें स्क्रीन में मौझूस सेटिंग्स आइकन को रिमोर्ट कंट्रोल के अप, डाउन और राइट, लेफ्ट बटन के जरीए सेलेक्ट करें और ओके का बटन प्रेस करें डिवाइस प्ektर पर क्लिक करें रिसेट के ऑप्शन क Fighter रिसाइट पर क्लिक करने के बाद इस एवरीथिंग ऑप्शन को स्लेक्ट करें समझते है according to Priyad


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 स्मार्ट एलेडी टीवी How to operate LED TV without remote control LED TV के पीछे मौझूद कंट्रोल बटन के जरिये आप अपने LED TV को कंट्रोल कर सकते हैं अगर TV स्टेंड बाय मोट पर है तो TV को आउन करने के लिए कंट्रोल बटन को दबाईए LED TV स्विच्च आउन होने की सूरत में कंट्रोल बटन को हलका प्रेस करके सॉर्स लिस को ओपन करें सॉर्स लिस में अपनी जरूरत के मताबग ओप्शन को सेलेक करने के लिए कंट्रोल बटन कोLeft या Right पुष्च करके कंट्रोल बटन को बीच में हलका प्रेस करें Source list कोaccix Roberts के लिए, Control बटन को देर तक टैक करें TV Volume को बढ़ाने या कम करना चाहिए Control बटन को Right या Left की तरफ पुष्ड करें चैनल को चेंज करने के लिए, ऊंपर यदि नीचे की तरफ Control बटन को पुष्ठ करें


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 स्मार्ट एलेडी टीवी पिच्चर स्क्रीन कटिंग फ्रॉम साइट कॉर्नर्स अगर आपकी स्क्रीन कॉर्नर्स से कट रही हैं, तो रिमोर्ट कंट्रोल में ओके के बटन को प्रेस करें कुइक बटन मेन्यू, डिस्प्ले स्क्रीन में आ जाए stab स्क्रीन मोड ऑप्शन को सिलेक्ट करें aspect ratio को auto से हटा कर 16-9 ऑप्शन पे सेट करें अब आपकी स्क्रीन फुल स्क्रीन आ रही होगी
[{'audio_file': 'Dawlance _ How to Select Input Sources By Remote Control.mp3', 'transcription': ' स्मार्ट एलेडी टीवी How to select input sources by remote control Remote control में मौझूद source बटन को क्लिक करें Source menu screen पर appear हो जाएगा मौझूद options के features को जानें Home इस option के जरिये आप YouTube, Netflix, Prime Video तमाम Android apps और Media Player for USB access को use कर सकते हैं ATV इस feature के जरिये आप लगाएगाई cable connection के थूँ analog channels देख सकते हैं DVB-T, T2 जिसमें terrestrial channels को देख सकते हैं ये feature पाकिस्तान में मौझूद नहीं है DVBC के जरिये आप लगाएगाई cable connection के थूँ digital channels देख सकते हैं sm